In [1]:
import pandas as pd
import numpy as np
import nest_asyncio
import asyncio
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from utils import read_pcap_file, combine_request_reply

nest_asyncio.apply()  # Needed for running the async function in the notebook


pd.set_option('display.max_colwidth', None)  # No truncation of column content

# PCAP files

In [2]:
dir_contents = [i for i in os.listdir("./data") if ".pcap" in i]
dir_contents

['sessionfailureEMS.pcap',
 'EMSSessionNoResp.pcap',
 'EMSSession11.pcap',
 'EMSSecurityAccessFailure.pcap',
 'PreconditionCheckFailA2.pcap',
 'Endless78.pcap',
 'vinfailure_continue.pcap',
 'EMSSession14.pcap',
 'PreconditionCheckNegResp.pcap',
 'EMSSession12.pcap',
 'EMSSession13.pcap',
 'vinfailure_stop.pcap']

# Read PCAP, Convert DataFrame

In [5]:
file_path = os.path.join("data", 'sessionfailureEMS.pcap')

df = asyncio.run(read_pcap_file(file_path))

df.head()


,number,timestamp,source,target,request,sid,error
0,138,2024-09-12 16:50:27,0x0e00,0x1032,True,0x3E,N/A
1,141,2024-09-12 16:50:27,0x1032,0x0e00,False,0x7E,N/A
2,144,2024-09-12 16:50:27,0x0e00,0x1032,True,0x3E,N/A
3,147,2024-09-12 16:50:27,0x1032,0x0e00,False,0x7E,N/A
4,149,2024-09-12 16:50:27,0x0e00,0x1032,True,0x22,N/A


# Merge Request and Reply into One Row

In [6]:
df = combine_request_reply(df)
df

,request_sid,reply_sid,error
0,0x3E,0x7E,N/A
1,0x3E,0x7E,N/A
2,0x22,0x7F,0x10
3,0x10,0x50,N/A
4,0x10,0x50,N/A
5,0x10,0x50,N/A
6,0x10,0x62,N/A
7,0x22,0x62,N/A
8,0x22,0x62,N/A
9,0x3E,0x7E,N/A
